In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import os
from tqdm import tqdm
from sklearn import linear_model
import helper_functions as hf 
from sklearn import linear_model


repo_path = '/home/tudor/Documents/GitHub/building_resilience'
data_path = repo_path + '/data/top_to_bottom/data'

In [2]:
## read data files:
data_path = repo_path + '/data/top_to_bottom/data'

# country codes for countries written in the UN format
CCs = pd.read_csv(data_path + '/help_data/UN_country_code.csv')
CCs = CCs.dropna(axis = 0).reset_index(drop=True)

# country codes for countries written in the WB format
WBs = pd.read_csv(data_path + '/help_data/WB_country_code.csv')
WBs = WBs.dropna(axis = 0).reset_index(drop=True)

#read WB data
WB_data = pd.read_csv(data_path+'/WB_WDI_data/wb_data.csv')

#read UN data
UN_data = pd.read_csv(data_path+'/UN_data/WPP2019_TotalPopulationBySex.csv')

# #read Climate reanalysis data
# climate_data = pd.read_csv(data_path+'/climate_data/era5_reanalysis.csv')

# #read GCM climate prediction data:
# prediction_data = pd.read_csv(data_path+'/climate_data/future_final.csv')

# read CDD data from Cimate reanalysis data and GCM prediction data:
CDD = pd.read_csv(data_path+'/climate_data/countrywise_CDDs.csv')
CDD = CDD.drop(columns=['Unnamed: 0'])
CDD = CDD.replace(0, np.nan)
CDD = CDD.dropna().reset_index(drop=True)

In [3]:
UN_prediction = hf.choose_UN_scenario(data_path,UN_data,'Medium')
UN_prediction=hf.drop_unwanted_UN_years(UN_prediction)

Medium scenario population prediction data succesfully extracted in your variable!


In [4]:
# initialise datasets for electricity:

industrial = pd.read_csv(data_path + '/help_data/WB_country_code.csv').dropna(axis = 0).reset_index(drop=True)
residential = pd.read_csv(data_path + '/help_data/WB_country_code.csv').dropna(axis = 0).reset_index(drop=True)
commercial = pd.read_csv(data_path + '/help_data/WB_country_code.csv').dropna(axis = 0).reset_index(drop=True)

for i in range(15):

    year = i+2000
    WB_dataset = hf.get_year_of_WB_data(year,WB_data)
    iea_path = data_path + '/IEA_data/IEA_percentage_breakdown_'+str(hf.IEA_year(year))+'.csv'
    IEA_dataset = pd.read_csv(iea_path)
    IEA_dataset = IEA_dataset.drop(['split','new_Data'], axis=1)

    WB_IEA = hf.merge_WB_and_IEA(WB_dataset,IEA_dataset)
    ## WorldBank has inputed thier NaN values as '..', so we need to remove them
    WB_IEA = WB_IEA.replace(to_replace='..', value=np.nan)
    WB_IEA = WB_IEA.dropna(axis=0).reset_index(drop=True)


    industry_elec_list = []
    residential_elec_list = []
    commercial_elec_list = []

    for i in range(len(WB_IEA)):
        df = WB_IEA.iloc[i,:]
        ind_elec = float(df.iloc[3])*float(df.iloc[5])*float(df.iloc[2])*float(df.iloc[6])*0.2
        industry_elec_list.append(ind_elec)
        res_elec = float(df.iloc[3])*float(df.iloc[5])*float(df.iloc[2])*float(df.iloc[7])*0.2
        residential_elec_list.append(res_elec)
        com_elec = float(df.iloc[3])*float(df.iloc[5])*float(df.iloc[2])*float(df.iloc[8])*0.2
        commercial_elec_list.append(com_elec)


    WB_IEA['Cooling industry electricity (kWh)'] = industry_elec_list
    WB_IEA['Cooling residential electricity (kWh)'] = residential_elec_list 
    WB_IEA['Cooling commercial electricity (kWh)'] = commercial_elec_list

    df1 = WB_IEA[['Country Code','Cooling industry electricity (kWh)']]
    industrial = industrial.merge(df1, on= 'Country Code', how = 'left')
    industrial = industrial.rename(columns={"Cooling industry electricity (kWh)": str(year)})
    
    df2 = WB_IEA[['Country Code','Cooling residential electricity (kWh)']]
    residential = residential.merge(df2, on= 'Country Code', how = 'left')
    residential = residential.rename(columns={"Cooling residential electricity (kWh)": str(year)})


    df3 = WB_IEA[['Country Code','Cooling commercial electricity (kWh)']]
    commercial = commercial.merge(df3, on= 'Country Code', how = 'left')
    commercial = commercial.rename(columns={"Cooling commercial electricity (kWh)": str(year)})

industrial = industrial.dropna(thresh = 4).reset_index(drop=True)
residential = residential.dropna(thresh = 4).reset_index(drop=True)
commercial = commercial.dropna(thresh = 4).reset_index(drop=True)


WB data for the year 2000 was succesfully imported!
WB data for the year 2001 was succesfully imported!
WB data for the year 2002 was succesfully imported!
WB data for the year 2003 was succesfully imported!
WB data for the year 2004 was succesfully imported!
WB data for the year 2005 was succesfully imported!
WB data for the year 2006 was succesfully imported!
WB data for the year 2007 was succesfully imported!
WB data for the year 2008 was succesfully imported!
WB data for the year 2009 was succesfully imported!
WB data for the year 2010 was succesfully imported!
WB data for the year 2011 was succesfully imported!
WB data for the year 2012 was succesfully imported!
WB data for the year 2013 was succesfully imported!
WB data for the year 2014 was succesfully imported!


In [5]:
intersect_country = pd.read_csv(data_path + '/help_data/WB_country_code.csv')

cdd_count = []
pop_count = []
elec_count= []

for i in range(len(intersect_country)):
    code = intersect_country.iloc[i]['Country Code']
    
    if (len(CDD[CDD['Country Code'] == code]) == 1):
        cdd_count.append(1)
    else:
        cdd_count.append(np.nan)
        
    if (len(UN_prediction[UN_prediction['Country Code'] == code]) == 1):
        pop_count.append(1)
    else:
        pop_count.append(np.nan)
        
    if (len(industrial[industrial['Country Code'] == code]) == 1):
        elec_count.append(1)
    else:
        elec_count.append(np.nan)
        
intersect_country['CDD'] = cdd_count
intersect_country['pop'] = pop_count
intersect_country['elec']=elec_count

all_intersect_country = intersect_country
intersect_country = intersect_country.dropna().reset_index(drop=True)


In [6]:
coeffs_df = pd.DataFrame({'Country Code':list(intersect_country['Country Code'])})

In [7]:
ind_c_0 = []; ind_c_1 = []; ind_c_2 = []
com_c_0 = []; com_c_1 = []; com_c_2 = []
res_c_0 = []; res_c_1 = []; res_c_2 = []
ind_pred= []; com_pred= []; res_pred= []

#### THIS IS WHERE THE FOR LOOP STARTS
for i in tqdm(range(len(intersect_country))):

    test_code = intersect_country['Country Code'].iloc[i]
    cdd_list = CDD[CDD['Country Code'] == test_code].iloc[0,2:17]
    pop_list = UN_prediction[UN_prediction['Country Code'] == test_code].iloc[0,2:17]
    ind_list = industrial[industrial['Country Code'] == test_code].iloc[0,2:17]
    com_list = commercial[commercial['Country Code'] == test_code].iloc[0,2:17]
    res_list = residential[residential['Country Code'] == test_code].iloc[0,2:17]
    dic_init = {'cdd':cdd_list, 'pop':pop_list, 'ind':ind_list, 'com':com_list, 'res':res_list}
    df_init = pd.DataFrame(dic_init)
    df_init = df_init.dropna().reset_index(drop=True)


    cdd_fut = CDD[CDD['Country Code'] == test_code].iloc[0,20:30]
    pop_fut = UN_prediction[UN_prediction['Country Code'] == test_code].iloc[0,23:33]
    dic_fut = {'cdd':cdd_fut, 'pop':pop_fut}
    df_fut = pd.DataFrame(dic_fut)
    df_fut = df_fut.dropna().reset_index(drop=True)
    
    lm_ind = linear_model.LinearRegression(normalize = True)
    X = df_init[['cdd','pop']]
    y_ind = df_init['ind']
    model = lm_ind.fit(X,y_ind)
    ind_c_0.append(lm_ind.intercept_); ind_c_1.append(lm_ind.coef_[0]); ind_c_2.append(lm_ind.coef_[1])
    ind_pred.append(lm_ind.predict(df_fut))
    
    lm_com = linear_model.LinearRegression(normalize = True)
    X = df_init[['cdd','pop']]
    y_com = df_init['com']
    model = lm_com.fit(X,y_com)
    com_c_0.append(lm_com.intercept_); com_c_1.append(lm_com.coef_[0]); com_c_2.append(lm_com.coef_[1])
    com_pred.append(lm_com.predict(df_fut))

    lm_res = linear_model.LinearRegression(normalize = True)
    X = df_init[['cdd','pop']]
    y_res = df_init['res']
    model = lm_res.fit(X,y_res)
    res_c_0.append(lm_res.intercept_); res_c_1.append(lm_res.coef_[0]); res_c_2.append(lm_res.coef_[1])
    res_pred.append(lm_res.predict(df_fut))

    
coeffs_df['inter_ind'] = ind_c_0 
coeffs_df['cdd_ind'] = ind_c_1
coeffs_df['pop_ind'] = ind_c_2
coeffs_df['inter_com'] = com_c_0
coeffs_df['cdd_com'] = com_c_1
coeffs_df['pop_com'] = com_c_2
coeffs_df['inter_res'] = res_c_0
coeffs_df['cdd_res'] = res_c_1
coeffs_df['pop_res'] = res_c_2    

100%|██████████| 114/114 [00:03<00:00, 36.01it/s]


In [11]:
industrial_pred = pd.DataFrame({'Country Code':list(intersect_country['Country Code'])})
commercial_pred = pd.DataFrame({'Country Code':list(intersect_country['Country Code'])})
residential_pred = pd.DataFrame({'Country Code':list(intersect_country['Country Code'])})
dum = [0]*len(industrial_pred)
for i in range(10):
    industrial_pred[str(2021+i)] = dum
    commercial_pred[str(2021+i)] = dum
    residential_pred[str(2021+i)]= dum
    
for i in range(len(industrial_pred)):
    industrial_pred.iloc[i,1:] = ind_pred[i]
    commercial_pred.iloc[i,1:] = com_pred[i]
    residential_pred.iloc[i,1:] = res_pred[i]

In [12]:
# industrial_pred = hf.change_to_fips(repo_path,industrial_pred)
industrial_pred

,Country Code,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030
0,DZA,6.874476e+11,7.144832e+11,7.506741e+11,7.717710e+11,8.181222e+11,8.453454e+11,8.707169e+11,9.001843e+11,9.209954e+11,9.687140e+11
1,AGO,2.873745e+10,3.057782e+10,3.248484e+10,3.438653e+10,3.665907e+10,3.848721e+10,4.066450e+10,4.278649e+10,4.489476e+10,4.716415e+10
2,ARG,1.387419e+12,1.418986e+12,1.466520e+12,1.490389e+12,1.517827e+12,1.553252e+12,1.577905e+12,1.618254e+12,1.660038e+12,1.672606e+12
3,ARM,1.982996e+10,1.900157e+10,1.876749e+10,1.900137e+10,1.939035e+10,2.038601e+10,1.905476e+10,1.920651e+10,1.905725e+10,2.099939e+10
4,AUS,1.879555e+12,1.867196e+12,1.875254e+12,1.876303e+12,1.872180e+12,1.884276e+12,1.886163e+12,1.890713e+12,1.889916e+12,1.904307e+12
5,AUT,8.117735e+11,8.205234e+11,8.273035e+11,8.302512e+11,8.342560e+11,8.397614e+11,8.462968e+11,8.510481e+11,8.567311e+11,8.611224e+11
6,AZE,7.996248e+10,8.237020e+10,8.056786e+10,8.459457e+10,8.976548e+10,9.878805e+10,9.228758e+10,9.130705e+10,9.073976e+10,1.060653e+11
7,BLR,3.097386e+11,3.092063e+11,3.122294e+11,3.137239e+11,3.143723e+11,3.150508e+11,3.149372e+11,3.182529e+11,3.205358e+11,3.183781e+11
8,BEL,7.687056e+11,7.635311e+11,7.671228e+11,7.573259e+11,7.565824e+11,7.588019e+11,7.466352e+11,7.439492e+11,7.436216e+11,7.417594e+11
9,BEN,1.737808e+09,1.825642e+09,1.926663e+09,2.031236e+09,2.082968e+09,2.174323e+09,2.330508e+09,2.422991e+09,2.502633e+09,2.599871e+09


In [13]:
industrial_pred = hf.change_to_fips(repo_path,industrial_pred)
commercial_pred = hf.change_to_fips(repo_path,commercial_pred)
residential_pred = hf.change_to_fips(repo_path,residential_pred)

industrial_pred.to_csv(data_path +'/output/simple_regression/industrial_prediction.csv')
commercial_pred.to_csv(data_path + '/output/simple_regression/commercial_prediction.csv')
residential_pred.to_csv(data_path + '/output/simple_regression/residential_prediction.csv')

coeffs_df = hf.change_to_fips(repo_path, coeffs_df)
coeffs_df.to_csv(data_path + '/output/simple_regression/coefficients.csv')

In [14]:
total_prediction = industrial_pred+commercial_pred+residential_pred
total_prediction['Country Code'] = industrial_pred['Country Code']
total_prediction = hf.change_to_fips(repo_path, total_prediction)
total_prediction.to_csv(data_path + '/output/simple_regression/total_prediction.csv')

In [15]:
industrial_complete = industrial.merge(industrial_pred, on='Country Code', how='left')
industrial_complete = industrial_complete.dropna(thresh = 18).reset_index(drop=True)
industrial_complete = hf.change_to_fips(repo_path, industrial_complete)
industrial_complete.to_csv(data_path + '/output/simple_regression/industrial_complete.csv')

commercial_complete = commercial.merge(commercial_pred, on='Country Code', how='left')
commercial_complete = commercial_complete.dropna(thresh = 18).reset_index(drop=True)
commercial_complete = hf.change_to_fips(repo_path, commercial_complete)
commercial_complete.to_csv(data_path + '/output/simple_regression/commercial_complete.csv')

residential_complete = residential.merge(residential_pred, on='Country Code', how='left')
residential_complete = residential_complete.dropna(thresh = 18).reset_index(drop=True)
residential_complete = hf.change_to_fips(repo_path, residential_complete)
residential_complete.to_csv(data_path + '/output/simple_regression/residential_complete.csv')

In [16]:
industrial = hf.change_to_fips(repo_path,industrial)
commercial = hf.change_to_fips(repo_path,commercial)
residential  = hf.change_to_fips(repo_path,residential)


industrial.to_csv(data_path + '/output/past_data/industrial.csv')
commercial.to_csv(data_path + '/output/past_data/commercial.csv')
residential.to_csv(data_path + '/output/past_data/residential.csv')

In [18]:
future_CDDs = CDD[['Country Code', '2021', '2022','2023','2024','2025','2026','2027','2028','2029','2030']]
pop_2014 = UN_prediction[['Country Code','2014']]

only_climate = coeffs_df.merge(pop_2014, on='Country Code', how ='left')
only_climate = only_climate.merge(future_CDDs, on='Country Code', how = 'left')
only_climate = only_climate.dropna()

only_climate_industrial = pd.DataFrame(only_climate['Country Code'])
only_climate_commercial = pd.DataFrame(only_climate['Country Code'])
only_climate_residential = pd.DataFrame(only_climate['Country Code'])

for i in range(10):
    year = 2021 + i
    ind_list = []
    com_list = []
    res_list = []
    for j in range(len(only_climate)):
        ind_list.append(only_climate['2014'].iloc[j]*only_climate['pop_ind'].iloc[j] + only_climate['inter_ind'].iloc[j] + only_climate['cdd_ind'].iloc[j]*only_climate[str(year)].iloc[j])
        com_list.append(only_climate['2014'].iloc[j]*only_climate['pop_com'].iloc[j] + only_climate['inter_com'].iloc[j] + only_climate['cdd_com'].iloc[j]*only_climate[str(year)].iloc[j])
        res_list.append(only_climate['2014'].iloc[j]*only_climate['pop_res'].iloc[j] + only_climate['inter_res'].iloc[j] + only_climate['cdd_res'].iloc[j]*only_climate[str(year)].iloc[j])
        
    only_climate_industrial[str(year)] = ind_list
    only_climate_commercial[str(year)] = com_list
    only_climate_residential[str(year)] = res_list

In [19]:
industrial_complete_climate = industrial.merge(only_climate_industrial, on='Country Code', how='left')
industrial_complete_climate = industrial_complete_climate.dropna(thresh = 18).reset_index(drop=True)
industrial_complete_climate = hf.change_to_fips(repo_path, industrial_complete_climate)
industrial_complete_climate.to_csv(data_path + '/output/only_climate/industrial_complete_climate.csv')

commercial_complete_climate = commercial.merge(only_climate_commercial, on='Country Code', how='left')
commercial_complete_climate = commercial_complete_climate.dropna(thresh = 18).reset_index(drop=True)
commercial_complete_climate = hf.change_to_fips(repo_path, commercial_complete_climate)
commercial_complete_climate.to_csv(data_path + '/output/only_climate/commercial_complete_climate.csv')

residential_complete_climate = residential.merge(only_climate_residential, on='Country Code', how='left')
residential_complete_climate = residential_complete.dropna(thresh = 18).reset_index(drop=True)
residential_complete_climate = hf.change_to_fips(repo_path, residential_complete_climate)
residential_complete_climate.to_csv(data_path + '/output/only_climate/residential_complete_climate.csv')